<a href="https://colab.research.google.com/github/RishipriyaDeekonda/Smartbridge/blob/main/major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**importing the libraries**

In [2]:
import os
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array


**Loading the Dataset**

In [3]:
!pip install kaggle

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle


cp: cannot stat 'kaggle.json': No such file or directory


In [5]:
!kaggle datasets download -d elinachen717/municipal-solid-waste-dataset


Dataset URL: https://www.kaggle.com/datasets/elinachen717/municipal-solid-waste-dataset
License(s): CC0-1.0
  0% 0.00/4.81M [00:00<?, ?B/s]
100% 4.81M/4.81M [00:00<00:00, 52.3MB/s]


In [6]:
!unzip /content/municipal-solid-waste-dataset.zip


Archive:  /content/municipal-solid-waste-dataset.zip
  inflating: Dataset/Biodegradable Images/TEST_BIODEG_HFL_0.jpeg  
  inflating: Dataset/Biodegradable Images/TEST_BIODEG_HFL_1.jpeg  
  inflating: Dataset/Biodegradable Images/TEST_BIODEG_HFL_10.jpeg  
  inflating: Dataset/Biodegradable Images/TEST_BIODEG_HFL_100.jpeg  
  inflating: Dataset/Biodegradable Images/TEST_BIODEG_HFL_1000.jpeg  
  inflating: Dataset/Biodegradable Images/TEST_BIODEG_HFL_1004.jpeg  
  inflating: Dataset/Biodegradable Images/TEST_BIODEG_HFL_1005.jpeg  
  inflating: Dataset/Biodegradable Images/TEST_BIODEG_HFL_1007.jpeg  
  inflating: Dataset/Biodegradable Images/TEST_BIODEG_HFL_1008.jpeg  
  inflating: Dataset/Biodegradable Images/TEST_BIODEG_HFL_1009.jpeg  
  inflating: Dataset/Biodegradable Images/TRAIN.1_BIODEG_ORI_10035.jpg  
  inflating: Dataset/Biodegradable Images/TRAIN.2_BIODEG_ORI_10025.jpg  
  inflating: Dataset/Biodegradable Images/TRAIN.2_BIODEG_ORI_10043.jpg  
  inflating: Dataset/Biodegradable Im

**Set Path to the Dataset:**

In [7]:
dataset_dir = '/content/Dataset'
classes = os.listdir(dataset_dir)


**Create Output Directories for Train, Validation, and Test Sets:**

In [8]:
output_dir = 'output_dataset'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)


**Create Subdirectories for Each Class:**

In [9]:
for cls in classes:
    os.makedirs(os.path.join(output_dir, 'train', cls), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'val', cls), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test', cls), exist_ok=True)


**Split the Dataset:**

In [10]:
    class_dir = os.path.join(dataset_dir, cls)
    images = os.listdir(class_dir)
    print(cls, len(images))

    train_and_val_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    train_images, val_images = train_test_split(train_and_val_images, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2


Biodegradable Images 130


**Copy Images to Respective Directories**:

In [12]:
    for img in train_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(output_dir, 'train', cls, img))
    for img in val_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(output_dir, 'val', cls, img))
    for img in test_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(output_dir, 'test', cls, img))
print("Dataset split into training, validation, and test sets.")


Dataset split into training, validation, and test sets.


**Define Directory Paths:**

In [13]:
dataset_dir = '/content/output_dataset'
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')
test_dir = os.path.join(dataset_dir, 'test')


**Define Image Size**:

In [14]:
IMG_SIZE = (224, 224)  # Common size for pre-trained models


**Create ImageDataGenerator for Augmentation and Rescaling:**

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

**Load and Resize Images from Directories:**

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define ImageDataGenerators for resizing and augmenting the images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define a separate ImageDataGenerator for validation and test (no augmentation, only rescaling)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load and resize the images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary'  # Assuming binary classification
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Do not shuffle test data
)



Found 78 images belonging to 4 classes.
Found 26 images belonging to 4 classes.
Found 26 images belonging to 4 classes.


**Print Class Indices:**

In [18]:
print(train_generator.class_indices)
print(val_generator.class_indices)
print(test_generator.class_indices)


{'Biodegradable Images': 0, 'Recyclable Images': 1, 'Trash Images': 2, 'archive.zip': 3}
{'Biodegradable Images': 0, 'Recyclable Images': 1, 'Trash Images': 2, 'archive.zip': 3}
{'Biodegradable Images': 0, 'Recyclable Images': 1, 'Trash Images': 2, 'archive.zip': 3}


**DATA VISUALISATION**